In [1]:
# Importing required packages

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import time
import datetime
from datetime import datetime
import socket
import getpass

#required packages
import mysql.connector as connection
from functools import reduce

In [2]:
st = []
ed = []
du = []
date_range= []
v_host_name = []

pd.set_option('display.max_column', None)

start = datetime.now()
st.append(start)

## Final Function
def f_baseline_survey(p_start_date, p_end_date):
    
    try:
        v_visit_started_date = p_start_date
        v_visit_ended_date = p_end_date
        
        def f_connect_to_database(p_host, p_user, p_password, p_database):
            while True:
                try:
                    # Setup database connection
                    v_mydb = connection.connect(
                        host=p_host,
                        user=p_user,
                        password=p_password,
                        database=p_database,
                        use_pure=True
                    )
                    v_cursor = v_mydb.cursor()
                    print("Connected to the database\n")
                    return v_mydb, v_cursor

                except connection.Error as err:
                    print(f"Error: {err}")
                    print("\n retry !")
                    return None, None


        # Function to close the database connection
        def f_close_database_connection(p_mydb, p_cursor):
            try:
                if p_cursor is not None:
                    p_cursor.close()
                    print("Cursor closed.")
                if p_mydb.is_connected():
                    p_mydb.close()
                    print("Database connection closed.\n")
            except Exception as e:
                return f"An error occurred while closing the database connection: {e}"

        while True:
            v_host_input = input("Enter the host: ")
            v_user_input = input("Enter the username: ")
            v_password_input = getpass.getpass("Enter the password: ")
            v_database_input = input("Enter the database: ")
            v_mydb,v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

            if v_mydb is not None:

            #Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
                break

        # Query 1
        v_queries = {'Query1':'''SELECT pat.patient_id AS Patient_id,
                    pat.date_created AS Patient_creation_date,
                    u.username AS CHW_name
                    FROM patient pat
                    JOIN users u ON (u.user_id=pat.creator)
                    WHERE pat.voided=0
                    AND pat.date_created BETWEEN %s AND %s;''',
                     
            'Query2':'''SELECT pat.patient_id AS Patient_id,
                    MAX(pi.identifier) AS OpenMRS_id
                    FROM patient pat
                    JOIN patient_identifier pi ON (pat.patient_id=pi.patient_id)
                    WHERE pat.voided=0
                    AND pi.identifier_type = 3
                    AND pi.voided=0
                    AND pat.date_created BETWEEN %s AND %s
                    GROUP BY Patient_id;''',
                     
            'Query3':'''SELECT pat.patient_id AS Patient_id,
                    p.gender AS Gender,
                    p.birthdate AS Date_of_birth,
                    YEAR(CURDATE()) - YEAR(p.birthdate) AS Age
                    FROM patient pat
                    JOIN person p ON (pat.patient_id=p.person_id)
                    WHERE pat.voided=0
                    AND p.voided=0
                    AND pat.date_created BETWEEN %s AND %s;''',
                     
            'Query4':'''SELECT pat.patient_id AS Patient_id,
                    pn.given_name AS Given_name,
                    pn.middle_name AS Middle_name,
                    pn.family_name AS Family_name
                    FROM patient pat
                    JOIN person_name pn ON (pat.patient_id=pn.person_id)
                    WHERE pat.voided=0
                    AND pn.voided=0
                    AND pn.preferred=1
                    AND pat.date_created BETWEEN %s AND %s;''',
                     
            'Query5':'''SELECT pi.patient_id AS Patient_id,
                    MAX(l.city_village) as Location_village,
                    MAX(ls.name) AS Location_sanch,
                    MAX(ld.name) AS Location_district,
                    MAX(l.state_province) as Location_state
                    FROM patient_identifier pi
                    JOIN location l ON (pi.location_id=l.location_id)
                    JOIN location ls ON (l.parent_location=ls.location_id)
                    JOIN location ld ON (ls.parent_location=ld.location_id)
                    WHERE pi.voided=0
                    AND pi.identifier_type = 3
                    AND pi.date_created BETWEEN %s AND %s
                    GROUP BY Patient_id;''',
        
            'Query6':'''SELECT pat.patient_id AS Patient_id,
                    pa.value,
                    pa.person_attribute_type_id
                    FROM patient pat
                    JOIN person_attribute pa ON (pat.patient_id=pa.person_id AND pa.person_attribute_type_id IN (13,15,25,26,27,55,56,8,24))
                    WHERE pat.voided=0
                    AND pa.voided=0
                    AND pat.date_created BETWEEN %s AND %s;'''}
        
        # Here append all the quries
        #v_query2 = [q1,q2,q3,q4,q5,q6]

        def f_get_date_range_from_tables(p_mydb, p_cursor):
            try:
                if p_mydb.is_connected():
                    # SQL query to fetch the minimum and maximum dates from "visit" table
                    date_query = "SELECT MIN(date_Started) AS min_date, MAX(date_Started) AS max_date FROM visit WHERE voided = 0;"

                    p_cursor.execute(date_query)
                    v_date_results = p_cursor.fetchall()

                    if v_date_results:
                        v_min_date, v_max_date = v_date_results[0]

                        return v_min_date, v_max_date

                    else:
                        print("No data available in the tables.")
                        return None, None
                else:
                    print("Database connection is not opened.")
                    return None, None
            except Exception as e:
                print(f"An error occurred: {e}")


        # Connect to the database
        v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

        # Get the date range from the SQL tables
        v_min_table_date, v_max_table_date = f_get_date_range_from_tables(v_mydb, v_cursor)

        # Close the database connection
        f_close_database_connection(v_mydb, v_cursor)

        # Print the date range to inform the user
        #print(f"Date range available in the SQL table: {v_min_table_date} to {v_max_table_date}\n")

        # Function to execute a query with date input parameters
        def f_execute_query_with_params(p_mydb, p_cursor, p_query, p_params):
            try:
                if p_mydb.is_connected():
                    p_cursor.execute(p_query, p_params)
                    v_results = p_cursor.fetchall()
                    v_columns = [desc[0] for desc in p_cursor.description]
                    df = pd.DataFrame(v_results, columns=v_columns)

                    return df
                else:
                    print("Database connection is not opened.")
                    return None
            except Exception as e:
                print(f"An error occurred: {e}")
                return None
            
        # Creating a function for fetching the data
        def f_fetch_data1(p_query, p_params):
            v_dataframe = []
            try:
                v_params = p_params
                
                #connect database and cursor     
                v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)
                    
                # Execute each query with input parameters and store the output in a dictionary
                v_output_dataframes = {}
                for query_name, query in v_queries.items():
                    if query_name == 'Query6':
                        df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                        if df is not None:
                            # Pivot the data to get each attribute value into separate columns
                            pivoted_data = df.pivot(index='Patient_id', columns='person_attribute_type_id', values='value').reset_index()

                            # Rename columns for better understanding
                            pivoted_data.rename(columns={8: 'Telephone', 13: 'Education',
                                                          15: 'Occupation', 25: 'Bank_account',
                                                          26: 'Mobile_type', 27: 'Whatsapp',
                                                          55: 'HOH_relation', 56: 'Marital_status', 24: 'HH_id'}, inplace=True)

                            # Group by patient_id and aggregate values for each column
                            # The 'first' function simply selects the first value encountered within each group.
                            grouped_data = pivoted_data.groupby('Patient_id').agg({'Telephone': 'first', 'Education': 'first',
                                                                                   'Occupation': 'first', 'Bank_account': 'first',
                                                                                   'Mobile_type': 'first', 'Whatsapp': 'first',
                                                                                   'HOH_relation': 'first', 'Marital_status': 'first',
                                                                                   'HH_id': 'first'}).reset_index()
                            # Merge the grouped data with the original DataFrame on 'Patient_id'
                            merged_data = pd.merge(df['Patient_id'], grouped_data, on='Patient_id', how='inner')
                            
                            # Drop the duplicates
                            merged_data = merged_data.drop_duplicates()

                            v_dataframe.append(merged_data)
                            print(f'{query_name} Executed successfully...\n')
                        
                    else:
                        df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                        if df is not None:
                            v_dataframe.append(df)
                            print(f"{query_name} executed successfully.\n")

                # Close the database connection
                f_close_database_connection(v_mydb, v_cursor)

                '''# Merge all the data frames based on 'visit_id' using 'outer' join
                merged_df = None
                for df in v_output_dataframes.values():
                    if merged_df is None:
                        merged_df = df
                    else:
                        merged_df = pd.merge(merged_df, df, on='Visit_id', how='outer')'''
                
                if len(v_dataframe)==1:
                    return v_dataframe[0]
                elif len(v_dataframe) == 2:
                    # Merge DataFrames from the list
                    v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='Patient_id', how='outer')  # Merge the first two DataFrames
                    return v_data
                else:
                    v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='Patient_id', how='outer')
                    # Continue merging with the rest of the DataFrames
                    for v_df in v_dataframe[2:]:
                        v_data = pd.merge(v_data, v_df, on='Patient_id', how='left')
                    return v_data
                
            except Exception as e:
                print('In this function', e)

        # Funciton for filtering the data between two days
        def f_filter_data(p_start_date, p_end_date, p_query):
            v_queries = []

            v_start_date1 = p_start_date
            v_end_date1 = p_end_date

            for i in range(len(p_query)):
                v_dummy = p_query[i].format(v_start_date1, v_end_date1)
                v_queries.append(v_dummy)

            v_data = f_fetch_data1(v_queries)
            return v_data
        
        ## SQL query to fetch the minimum and maximum dates from "visit" table
        v_min_date = v_min_table_date
        v_last_date = v_max_table_date
        v_hours = v_min_date.hour
        v_minutes = v_min_date.minute
        v_seconds = v_min_date.second

        print(f'Date Between **Starting Date - ({v_min_date}) and **End Date - ({v_last_date}).\n\n**The format is yyy-mm-dd\n')

        try:
            while True:
                # Getting input from user
                v_strt_date_input = v_visit_started_date

                pd.to_datetime(v_strt_date_input, format='%Y-%m-%d')
                #v_start_date = v_strt_date_input+' '+str(v_hours)+':'+str(v_minutes)+':'+str(v_seconds)
                v_start_date = v_strt_date_input+' 00:00:00'
                date_format = '%Y-%m-%d %H:%M:%S'

                # Convert the string to a datetime object
                datetime_object = datetime.strptime(v_start_date, date_format)
                print("\nDate Entered: \t",datetime_object)

                try:
                    if (datetime_object<=v_last_date): # (datetime_object>=v_min_date) and 
                        while True:
                            # Getting input from user
                            v_end_date_input = v_visit_ended_date
                            pd.to_datetime(v_end_date_input, format='%Y-%m-%d')
                            v_end_date = v_end_date_input+' 23:59:59'

                            #v_end_date = v_end_date+' 00:00:00'
                            # Convert the string to a datetime object
                            datetime_object1 = datetime.strptime(v_end_date, date_format)
                            print("\nDate Entered: \t",datetime_object1,'\n')

                            try:
                                if (datetime_object1<v_last_date) or (datetime_object1>datetime_object):
                                    if datetime_object1 < datetime_object:
                                        print('Incorrct Date. Please Enter the Date within the range.')
                                    else:
                                        # Getting the username
                                        v_hostname = socket.gethostname()
                                        v_host_name.append(v_hostname)
                                        
                                        # Execute each query with input parameters
                                        v_params = (v_start_date, v_end_date)
                                                                                
                                        # Entered data ranges
                                        date_range.append(v_start_date+' / '+v_end_date)
                                        
                                        # Passing the startdate, end date and query in the below function to fetch the data
                                        v_sample = f_fetch_data1(v_queries, v_params)
                                        v_sample = v_sample.drop_duplicates()
                                        
                                        # Rearrange the dataframe columns as per the report file
                                        v_sample = v_sample[['Patient_id','OpenMRS_id','Location_village','Location_sanch','Location_district','Location_state','Patient_creation_date',
                                                   'CHW_name','Given_name','Middle_name','Family_name','Gender','Date_of_birth','Age','HH_id','Education',
                                                  'Occupation','Bank_account','Mobile_type','Whatsapp','HOH_relation','Marital_status','Telephone']].copy()
                                        
                                        print('\nBaseline Survey dataframe created\n')
                                        
                                        # Get the current date and time
                                        file_name = datetime.now()
                                        # Format the datetime object
                                        formatted_file_name = int(file_name.strftime("%Y%m%d%H%M%S"))

                                        # Conver the dataframe into an excel file
                                        v_sample.to_excel(f'Baseline_survey_report_{formatted_file_name}.xlsx', index=False)
                                        
                                        print(f'\nFinal Baseline_survey_report_{formatted_file_name}.xlsx Excel Output Created...:)\n')
                                        
                                        end = datetime.now()
                                        ed.append(end)

                                        duration = ed[-1]-st[-1]
                                        du.append(duration)

                                        dur = {'Host_name':v_host_name, 'Date_Range':date_range, 'Start': st, 'End':ed, 'Duration':du}
                                        duration_df = pd.DataFrame(dur)

                                        # Convert the 'Duration' column to seconds
                                        duration_df['Duration'] = duration_df['Duration'].dt.total_seconds()
                                        duration_df.to_excel('Duration2.xlsx', index=False)
                                        
                                        #print('Executed successfully ')
                                        
                                        # Return the file dataframe and the duration dataframe
                                        return v_sample, duration_df
                                        
                                else:
                                    print('Enter the date with range')
                            except Exception as e:
                                print('Error 2', e)
                                break
                        break
                    else:
                        print('Error3')
                    #break
                except Exception as e:
                            print(e)
        except Exception as e:
            print('Error 4', e)
    except Exception as e:
        print('Error in the main function: ', e)
        

### '2023-05-12' AND '2024-03-19'

In [3]:
v_base_survey = f_baseline_survey('2023-05-12', '2024-03-22')

Enter the host: afitraining.ekalarogya.org
Enter the username: root
Enter the password: ········
Enter the database: openmrs
Connected to the database

Cursor closed.
Database connection closed.

Connected to the database

Cursor closed.
Database connection closed.

Date Between **Starting Date - (2023-05-12 13:48:58) and **End Date - (2024-03-21 17:18:21).

**The format is yyy-mm-dd


Date Entered: 	 2023-05-12 00:00:00

Date Entered: 	 2024-03-22 23:59:59 

Connected to the database

Query1 executed successfully.

Query2 executed successfully.

Query3 executed successfully.

Query4 executed successfully.

Query5 executed successfully.

Query6 Executed successfully...

Cursor closed.
Database connection closed.


Baseline Survey dataframe created


Final Baseline_survey_report_20240322124817.xlsx Excel Output Created...:)



In [4]:
v_base_survey[0]

,Patient_id,OpenMRS_id,Location_village,Location_sanch,Location_district,Location_state,Patient_creation_date,CHW_name,Given_name,Middle_name,Family_name,Gender,Date_of_birth,Age,HH_id,Education,Occupation,Bank_account,Mobile_type,Whatsapp,HOH_relation,Marital_status,Telephone
0,481044,2AEVC-4,Anbe,Jogmudi,Nashik,Maharashtra,2023-05-12 13:53:53,nurse1,Test,None,Bhushan Offline Mode 1,M,1992-05-12,32,452db2ce-6252-42af-9883-09a0c7e77a5c,Passed secondary school (Class 10th boards),Tenant Farmer or Agricultural Daily Wage Worker,No,Own Feature Phone,Yes (Personal),Niece or Nephew,Married,9655747434
1,481045,2AEVD-1,Basanvahi,Charama,Kanker,Chhattisgarh,2023-05-12 14:46:53,CHBH0136bas,Chinta,Ram,Kawachi,M,1995-01-01,29,cd1dad14-3876-412d-8896-1fd441da26a9,Cannot read or write,Agricultural Farmer,Yes,None,Yes (Family Member),-,Married,7049373489
2,481046,2AEVE-9,Basanvahi,Charama,Kanker,Chhattisgarh,2023-05-12 14:46:53,CHBH0136bas,Puniya,Bai,Kawachi,F,1995-01-01,29,cd1dad14-3876-412d-8896-1fd441da26a9,Cannot read or write,Household Work,Yes,None,Yes (Family Member),Spouse,Married,7049373489
3,481047,2AEVF-7,Basanvahi,Charama,Kanker,Chhattisgarh,2023-05-12 14:46:53,CHBH0136bas,Devraj,None,Kawachi,M,1993-10-06,31,cd1dad14-3876-412d-8896-1fd441da26a9,Passed secondary school (Class 10th boards),Agricultural Farmer,No,Own Smartphone,Yes (Family Member),Son or Daughter,Unmarried,7049373489
4,481048,2AEVG-5,Basanvahi,Charama,Kanker,Chhattisgarh,2023-05-12 14:46:54,CHBH0136bas,Nathu,Ram,Kawachi,M,1999-02-28,25,cd1dad14-3876-412d-8896-1fd441da26a9,Passed senior secondary school (Class 12th boa...,Agricultural Farmer,No,Own Smartphone,Yes (Family Member),Son or Daughter,Unmarried,7049373489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10184,493198,2C6XH-2,Bharatnathri,Dumma,Deoghar,jharkhand,2024-03-21 23:20:54,arpannurse,Arpan,None,Hypertension Screening,M,2006-03-21,18,b60abfa9-01f5-4fbc-891d-ee781661467e,Passed secondary school (Class 10th boards),Salaried Government Job,Yes,None,Yes (Family Member),Grandparent or Grandparent-In-Law,Married,9123116015
10185,493199,2C6XJ-7,Jabla,Manbazar,Purulia,West Bengal,2024-03-22 09:43:26,nurse1,New,None,START,M,2024-03-22,0,877dd938-091d-4a76-9ec7-d2c230e3ec9c,Passed senior secondary school (Class 12th boa...,Agricultural Farmer,Yes,Shared Smartphone,Yes (Family Member),Brother or Sister,Unmarried,9566338852
10186,493200,2C6XK-5,baghmara,Dumma,Deoghar,jharkhand,2024-03-22 10:23:20,nurse1,Nishita,None,Test,F,1994-10-22,30,00f58463-041a-487e-a011-c75f45a4ed05,Can read and write,Salaried Government Job,Yes,Shared Feature Phone,Yes (Family Member),Father or Mother-In-Law,Unmarried,9737979976
10187,493201,2C6XL-3,Anbe,Jogmudi,Nashik,Maharashtra,2024-03-22 10:26:53,nurseqa,Test,None,Hypertension Scr.,M,1996-03-22,28,d228265f-b980-4a0f-bf72-b82dd44e5ec9,Graduation and above,Petty Business or Shop Owner,No,Own Smartphone,Yes (Personal),Son or Daughter,Unmarried,9665574743


In [5]:
df1 = v_base_survey[0].copy()

In [6]:
df1.duplicated().sum()

0

In [7]:
df1['Patient_id'].duplicated().sum()

0

In [8]:
duration = v_base_survey[1]

In [9]:
duration

,Host_name,Date_Range,Start,End,Duration
0,JOHAN-PC,2023-05-12 00:00:00 / 2024-03-22 23:59:59,2024-03-22 12:47:59.603603,2024-03-22 12:48:24.678842,25.075239
